## ReMKiT1D input generator - testing kinetic advection using a wave problem 

This example tests the kinetic advection operators by building a toy wave problem from the Vlasov advection terms for f0 and f1

This notebook corresponds to Section 6.2.1. od the ReMKiT1D code paper.

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.common_models as cm

### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_kin_adv_test/" 
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 4 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells

rk.setMPIData(numProcsX,numProcsH,haloWidth)

### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

### Grid setup

In [ ]:
xGrid = 0.1*np.ones(128) 
L = sum(xGrid)
vGrid = np.logspace(-2,1,80) #In normalized velocity - default normalization is thermal velocity sqrt(m_e * k * T_e/2)
lMax = 1
gridObj = Grid(xGrid,vGrid,lMax,interpretXGridAsWidths=True,isPeriodic=True)


In [ ]:
# Add the grid to the wrapper
rk.grid = gridObj

### Set default species

In [ ]:
rk.addSpecies("e",0)
rk.addSpecies("D+",-1,atomicA=2.014,charge=1.0)

### Variables

In [ ]:
n = np.exp(-(gridObj.xGrid-np.mean(gridObj.xGrid))**2) # A Gaussian spatial profile

f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numV()):
    f[:,gridObj.getH(0)-1,i] = n 
    
rk.addVar("f",f,isDistribution=True,isCommunicated=True)
rk.addVar("time",isScalar=True,isDerived=True)

### Adding the advection model

The spatial advection model is implemented in common_models.py and is just used directly here. For implementation details see the corresponding function.

In [ ]:
advModel = cm.kinAdvX(modelTag="adv",distFunName="f",gridObj=gridObj)
rk.addModel(advModel.dict())

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=10.0) #Everything default except for more lenient absolute convergence tolerance

rk.addIntegrator("BE",integrator)

Set initial timestep length and numbers of allowed implicit and general groups

In [ ]:
initialTimestep =  0.01

rk.setIntegratorGlobalData(1,1,initialTimestep) 

A single integration step

In [ ]:
bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1",bdeStep.dict())

#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(4000)
rk.setFixedStepOutput(40)

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
numFiles = 100

#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

In [ ]:
loadedData.coords['x'].attrs['units'] = "$x_0$"
loadedData.coords['v'].attrs['units'] = "$v_{th}$"
loadedData.coords['time'].attrs['standard_name'] = 't'
loadedData.coords['time'].attrs['units'] = "$t_0$"

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Compare to analytic travelling wave solution

In [ ]:
wave_speeds=gridObj.vGrid/np.sqrt(3)
f0_analytic=np.zeros((numFiles+1,gridObj.numX(),gridObj.numV()))
times = loadedData.coords['time'].data

for i in range(numFiles+1):
    for k,c in enumerate(wave_speeds):
        leftPositionMod = (gridObj.xGrid-c*times[i]) % L
        leftPosition = np.where(leftPositionMod > 0,leftPositionMod,leftPositionMod+L)
        rightPosition = (gridObj.xGrid+c*times[i]) % L
        f0_analytic[i,:,k] = 0.5*(np.exp(-(leftPosition-np.mean(gridObj.xGrid))**2) + np.exp(-(rightPosition-np.mean(gridObj.xGrid))**2))


In [ ]:
dataName = 'f'
vInd = 40 # Lower velocities will be less diffusive
curveDict = {t: hv.Scatter(loadedData[dataName][{"time":t,"h":0,"v":vInd}]).opts(marker="o",color="r",s=6.0)*hv.Curve((gridObj.xGrid,f0_analytic[t,:,vInd])) for t in range(numFiles+1)}
kdims = [hv.Dimension(('time', 'Time'),unit=loadedData.coords["time"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims)

### Plot for the paper

In [ ]:
diff = np.abs(f0_analytic - loadedData['f'][:,:,0,:])

In [ ]:
plotVPoints = [0,19,39,59]

In [ ]:
relativeErrorPlot=hv.Overlay([hv.Curve(diff.reduce(np.max,'x')[:,v],label="$v={:.2f}$".format(rk.grid.vGrid[v])+'$ v_{th}$').opts(ylabel='$\Delta f_0$',linestyle='--',fontscale=1.5, fig_size=150,linewidth=2.0,logy=True,ylim=(1e-7,1)) for v in plotVPoints]).opts(legend_position='top',legend_cols=2)

In [ ]:
hv.output(fig='pdf')
hv.save(relativeErrorPlot, 'advectionKinTestRelErr.pdf', dpi=144)